# Spytial

Sometimes you just want to see your data.

You’re working with a tree, a graph, a recursive object -- maybe an AST, a neural network, or a symbolic term. You don’t need an interactive dashboard or a production-grade visualization system. You just need a diagram, something that lays it out clearly so you can understand what’s going on.

That’s what `sPyTial` is for. It’s designed for developers, educators, and researchers who work with structured data and need to make that structure visible — to themselves or to others — with minimal effort. 



In [28]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import diagram
from spytial.annotations import orientation, attribute, hideAtom, atomColor, group, flag


## Python Gives You Structure — But Not Visuals
Python is great at structure. When you build a tree, a graph, or any recursive data structure, the relationships are right there — through fields and references.

Let’s define a simple binary tree, construct an instance of the tree, and print it.

In [29]:
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __repr__(self):
        return f"TreeNode({self.value})"

# Instance of a binary tree
root = TreeNode(
    10,
    left=TreeNode(
        5,
        TreeNode(3),
        TreeNode(7)
    ),
    right=TreeNode(
        15,
        TreeNode(12),
        TreeNode(18)
    )
)

print(root)

TreeNode(10)


You only see the root. The rest of the tree is hidden behind references. 

To make structure visible, you might write a recursive print function:

In [30]:
def print_tree(node, level=0):
    if node.right:
        print_tree(node.right, level + 1)
    print("    " * level + str(node.value))
    if node.left:
        print_tree(node.left, level + 1)

print_tree(root)

        18
    15
        12
10
        7
    5
        3


This is better than a bare `print` statement, and gives you some idea of the shape of the tree. You could imagine writing an even more involved visualization that ensures that things are drawn even better. But fundamentally, this approach is:

- Hardcoded for trees (and / or any other structure you envision)
- Constructive: You get nothing until you have written all the code for your visualization.
- Tedious to write and debug.


#### Or You Can Use sPyTial
With sPyTial, you don’t have to write a printer at all. You can generate a box-and-arrow diagram automatically.

You'll see a diagram that:

- Draws one box per object
- Connects objects via their fields (like .left, .right)
- Labels boxes with class names and values

This works out-of-the-box — no layout rules, no walkers, no extra code.

In [31]:
from spytial import diagram

diagram(root)


To generate the diagram, sPyTial performs relationalization. That is, it turns your object into a graph.

Conceptually, your data becomes:

- Atoms: each Python object becomes a node
- Relations: each reference (like x.left = y) becomes an edge

It’s the same transformation you’d do manually if you were building a visualizer — but done automatically, for any Python object.

## Adding Spatial Semantics with Constraints 

sPyTial gives you a default diagram out-of-the-box. But the layout isn’t always what you mean. Take our tree example. The diagram shows the objects and their connections — but the children might appear side-by-side, above the parent, or in arbitrary positions. That’s because you haven’t told sPyTial what the structure means spatially.


Think about how you picture a binary tree:

- The root is at the top.
- The left child is below and to the left.
- The right child is below and to the right.

That’s not just data, it expresses something semantic about how you would like the tree laid out. 
With sPyTial, you can express that layout intent directly, using decorators.

In [32]:
@orientation(selector='{ x : TreeNode, y : TreeNode | x.left = y}', directions=['below', 'left'])
@orientation(selector='{ x : TreeNode, y : TreeNode | x.right = y}', directions=['below', 'right'])
@attribute(field='value')
@hideAtom(selector='NoneType')
@flag(name="hideDisconnected")
class TreeNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
    
root = TreeNode(
    value=10,
    left=TreeNode(
        value=5,
        left=TreeNode(3),
        right=TreeNode(7)
    ),
    right=TreeNode(
        value=15,
        left=TreeNode(12),
        right=TreeNode(18)
    )
)

Lets break these annotations down:

- `@orientation(...)` defines how related elements should be positioned.
   - The first annotation says: "If x.left = y, then place y below and to the left of x."
   - The second: "If x.right = y, then place y below and to the right of x."

These aren’t just hints — they’re enforced by the layout engine.

- `@attribute(field='value')` tells sPyTial to label each box with the value field, so instead of seeing TreeNode, you’ll see the number stored in that node.

- `@hideAtom(selector='NoneType')` prevents None references (like empty children) from appearing in the diagram.

- `@flag(name="hideDisconnected")` tells sPyTial to hide objects that aren’t connected to anything — useful if you have unused or floating nodes in memory.


So now, when you diagram `root`, all these annotations are enforced.

In [33]:
diagram(root)

##  Red-Black Trees with Color Coding
So far, we’ve visualized structure and layout. But many data structures carry semantic metadata that you also want to reflect in a diagram — things like types, states, or properties.

A perfect example is the Red-Black Tree.

Red-Black Trees augment binary search trees with a color field that encodes balancing invariants. Those colors are essential to understanding the shape and behavior of the structure.

We’ll subclass TreeNode to add a color field and annotate it using @atomColor, so that red and black nodes are drawn differently.

In [34]:
from spytial import atomColor

@atomColor(
    selector='{ x : RBTreeNode | @:(x.color) = red }',
    value='red'
)
@atomColor(
    selector='{ x : RBTreeNode | @:(x.color) = black }',
    value='black'
)
@attribute(field='color')
class RBTreeNode(TreeNode):
    def __init__(self, value, color, left=None, right=None):
        self.color = color
        super().__init__(value, left, right)


### What’s Going On?
`@atomColor(...)` uses a selector to match nodes based on a field (in this case, color).

`@:(x.color)` refers to the value of a color node, rather than its unique atom id. Matching nodes are assigned a color (red, black) in the diagram.


In [35]:
rb_root = RBTreeNode(
    value=10, color="black",
    left=RBTreeNode(
        value=5, color="red",
        left=RBTreeNode(3, "black"),
        right=RBTreeNode(7, "black")
    ),
    right=RBTreeNode(
        value=15, color="red",
        left=RBTreeNode(12, "black"),
        right=RBTreeNode(18, "black")
    )
)


diagram(rb_root)

## When Layout Fails: Unsatisfiable Spatial Constraints
sPyTial doesn't just generate diagrams — it enforces your spatial intent. When your annotations conflict — when there’s no way to satisfy all the constraints you wrote — it lets you know:

⚠️ This layout is unsatisfiable. Something doesn’t add up.

That’s what sets sPyTial apart: it treats spatial layout like a specification, not just a drawing.

### When Your Data Isn’t a TreeNode
So far, we’ve assumed that the data matches the structure implied by our annotations.

But what if it doesn’t?

Let’s annotate a class with tree semantics — a parent node with left and right children, laid out spatially.

Then we’ll feed it bad data — data that violates those semantics (e.g., a child with multiple parents).

sPyTial won’t just draw the diagram — it will tell us the layout is unsatisfiable.

In [36]:

shared = TreeNode(99)

not_a_tree = TreeNode(
    1,
    left=TreeNode(2, left=shared, right = TreeNode(11)),  # 🚨 shared as left
    right=shared  # 🚨 shared as both left and right
)


diagram(not_a_tree)



## TODO: Cyclic Constraints

## Object-Level Annotations

So far, we’ve attached annotations to classes — but what if you want to annotate a specific object?

This comes up all the time:
- You don’t want to (or can’t) modify a class definition
- You’re visualizing different objects of the same type in the same diagram
- You want to group the contents of one set, but not another


sPyTial supports object-level annotations using function decorators. **All annotations are available as function decorators and vice-versa**.

### Demo: Grouping Different Sets

Let’s create three Python set objects:

In [ ]:
fruits = {"apple", "banana", "cherry", "date"}
numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
colors = {"red", "green", "blue"}


Each set contains strings or integers. We'll visualize all three — but only use the group annotation on two of them.


In [37]:
from spytial import group

# Group members of 'fruits' under the fruits set
fruits = group(
    field='contains',         # Which relation to group
    selector='self',          # Apply this annotation to this object only
    groupOn=0,                # Group based on the set (position 0 in the tuple)
    addToGroup=1              # Add the elements (position 1) into that group
)(fruits)

# Same for numbers
numbers = group(
    field='contains',
    selector='self',
    groupOn=0,
    addToGroup=1
)(numbers)

# Leave 'colors' ungrouped


This will cause the diagram to place all fruit elements inside a visual "fruits" group, do the same for numbers, and render colors as flat nodes without grouping

In [38]:
diagram({
    "fruits": fruits,
    "numbers": numbers,
    "colors": colors
})


# Relationalizers / Providers


While `sPytial` makes a best effort to relationalize most data structures into a set of atoms (i.e. nodes) and relations (i.e. edges), you may want to customize how this happens for a specific domain.

This is what **providers** are designed to do. By defining how Python objects become atoms and relations, you can:
- Emphasize relationships important relationships to your domain of choice.
- Control visualization granularity and detail.


In [39]:
from spytial import diagram
from spytial.provider_system import RelationalizerBase, relationalizer, Atom, Relation
from typing import Any, List, Tuple
from datetime import datetime, date

# Example: Custom provider for datetime objects

## Higher priority means your provider is considered before other providers.
## Built in providers reserve priority up to 99.
@relationalizer(priority=101)
class DateTimeProvider(RelationalizerBase):
    """Custom provider for datetime objects with temporal relationships."""
    
    def can_handle(self, obj: Any) -> bool:
        """Return true if your provider can relationalize the object passed in."""
        return isinstance(obj, (datetime, date))
    
    def relationalize(self, obj: Any, walker_func) -> Tuple[Atom, List[Relation]]:
        """
        Return an Atom and list of Relations (not dicts and tuples).
        """
        obj_id = walker_func._get_id(obj)
        
        if isinstance(obj, datetime):
            label = obj.strftime("%Y-%m-%d %H:%M:%S")
            atom_type = "datetime"
        else:
            label = obj.strftime("%Y-%m-%d")
            atom_type = "date"
        
        # Create proper Atom object
        atom = Atom(
            id=obj_id,
            type=atom_type,
            label=label
        )
        
        # No relations for simple datetime objects
        relations = []
        
        return [atom], relations

# Test the datetime provider
temporal_data = {
    'created': datetime(2024, 1, 15, 10, 30, 0),
    'updated': datetime(2024, 1, 16, 14, 45, 0),
    'launch_date': date(2024, 2, 1)
}

diagram(temporal_data)